In [1]:
#实际上，以最大度节点为起点进行迭代是有问题的 是不是应该改为以 能到达最大度节点的最远的那个节点为起始点？


In [2]:
import openpyxl
import copy
from openpyxl import Workbook
import json
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

In [3]:
def get_values(filename,id = 0,delme = 0):
    wb = openpyxl.load_workbook(filename)
    sheet_names = wb.sheetnames
    print(sheet_names)

    sheet = wb[sheet_names[id]]
    maxrow = sheet.max_row 
    maxcol = sheet.max_column 
    value = sheet.cell(row=2, column=1).value
    alldata = []

    for row in sheet.iter_rows(min_row=1, max_row=maxrow, min_col=1, max_col=maxcol): # type: ignore
        row_data = [cell.value for cell in row]
        alldata.append(row_data)
    station_name = alldata[0][1:]

    if delme:
        del alldata[0]
        for i in range(len(alldata)):
            del alldata[i][0]


    return alldata,station_name

In [4]:
def refineme(data):
    for i in range(len(data)):
        for j in range(len(data[i])):
            if data[i][j] is None:
                data[i][j] = 0
    return data

In [5]:
def convme(data,title):
    degree = {}
    alldegree = 0
    for i,row in enumerate(data):
        
        degree[title[i]] = sum(row)
        alldegree = alldegree + sum(row)
    return degree,alldegree

In [6]:
def findme_maxdegreee(degree):
    max_degree = max(degree.values())

    max_degree_node = []
    for key in degree:
        if int(degree[key]) == int(max_degree):
            max_degree_node.append(key)
    return(max_degree,max_degree_node)

In [7]:
def picme(station_name,MyOD):
    plt.figure(figsize=(10, 6))
    MyOD = np.array(MyOD)
    plt.imshow(MyOD, cmap='Blues', aspect='auto', interpolation='nearest')
    plt.colorbar(label='Value')
    plt.xticks(ticks=np.arange(len(station_name)), labels=station_name, rotation=45)
    plt.yticks(ticks=np.arange(MyOD.shape[0]), labels=station_name)
    plt.title('Graph Adjacency')
    plt.xlabel('Columns')
    plt.ylabel('Rows')
    plt.tight_layout()
    plt.show()

In [8]:
def dfs(matrix, start):
    vis_his = []  # 访问历史记录
    visited = []  # 已访问节点列表
    stack = [start]  # 用栈实现深度优先搜索
    vis_his.append(start)

    while stack:
        #print(stack)
        node = stack.pop()  # 弹出栈顶节点
        if node not in visited:
            visited.append(node)  # 标记为已访问
            # 将与当前节点相连的未访问节点压入栈中
            for i in range(len(matrix[node]) - 1, -1, -1):  # 从后往前遍历，确保顺序一致
                if matrix[node][i] == 1 and i not in visited:
                    stack.append(i)
                    vis_his.append(i)
    print(vis_his)
    vis_his = list(set(vis_his))
    return visited, vis_his

# BFS函数
def bfs(matrix, start):
    vis_his = []
    visited = []  # 已访问节点列表
    queue = [start]  # 待访问节点队列
    vis_his.append(start)
    while queue:
        #print(queue)
        node = queue.pop(0)  # 弹出队列头部节点
        if node not in visited:
            visited.append(node)  # 标记为已访问
            # 将与当前节点相连的未访问节点加入队列
            for i in range(len(matrix[node])):
                if matrix[node][i] == 1 and i not in visited:
                    queue.append(i)
                    vis_his.append(i)
    print(vis_his)
    vis_his = list(set(vis_his))
    return visited,vis_his

In [9]:
def replace_ones(matrix, i):
    # 替换第i行的1为0
    matrix[i] = [0 if x == 1 else x for x in matrix[i]]
    
    # 替换第i列的1为0
    for row in matrix:
        if row[i] == 1:
            row[i] = 0

In [10]:
MyOD,station_name = get_values('OD.xlsx',delme=1)

['Sheet1']


In [11]:
station_index = {}
index_station = {}
for i , item in enumerate(station_name):
    station_index[item] = i
    index_station[i] = item

In [12]:
MyOD = refineme(MyOD)
i = 0

In [13]:
def dropme(MyOD,allready_vis):
    for item in allready_vis:
        # 遍历每一行
        MyOD[station_index[item]] = [0 if x == 1 else x for x in MyOD[station_index[item]]]
        for row in MyOD:
            if row[station_index[item]] == 1:
                row[station_index[item]] = 0
    return MyOD

In [14]:
degree = convme(MyOD,station_name)

In [15]:
startpoint = []

In [16]:
i = 0  # 初始化i
visrec = []
startpoint = []  # 初始化起点列表
again_visit = copy.deepcopy(station_name)  # 未访问的所有节点列表

while True:
    print(f'Turn {i+1}')
 
    # 调用 convme 函数，获取度数和是否存在有效度数
    degree, alldegree = convme(MyOD, station_name)
    
    # 检查是否有有效的度数
    if alldegree:
        #picme(station_name, MyOD)
        max_degree, max_degree_node = findme_maxdegreee(degree)
        print(max_degree_node)
        
        # 获取最大度数的节点并添加到起点列表
        mychooice = np.random.randint(0,len(max_degree_node))
        print(mychooice)
        startpoint.append(max_degree_node[mychooice])
        print(f'The max degree is {max_degree} @ {max_degree_node[mychooice]}')

        visrec.append(max_degree_node[mychooice])
        
        # 调用 bfs 函数，获取访问的节点（bfs_order 是节点的索引）
        bfs_order, allready_vis = bfs(MyOD, station_index[max_degree_node[mychooice]])
        
        # 更新已访问的节点列表，转换为节点名称
        allready_vis = [index_station[item] for item in bfs_order]
        print(f'The turn {i+1} visited node is {allready_vis}')
        
        # 从 MyOD 中删除已访问的节点
        MyOD = dropme(MyOD, allready_vis)
        
        # 调试：确保 MyOD 中已删除的节点
        #print(f'MyOD after drop: {MyOD}')
        
        # 更新仍未访问的节点列表，排除已访问的节点
        again_visit = [item for item in again_visit if item not in allready_vis]
        
        # 调试：输出仍未访问的节点
        print(f'Still node {again_visit} not visited')
        print(f'-----------------------------------------------------')
        
    else:
        # 当没有更多的度数时，合并已访问的节点
        print(f'-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+')
        startpoint = startpoint + again_visit
        # 按照数字部分排序
        sorted_startpoint = sorted(startpoint, key=lambda x: int(x[1:]))


        print(f'Total node {sorted_startpoint} is the origin. \n {len(sorted_startpoint)} nodes')
        print(f'+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-')
        break  # 结束循环
    
    i += 1  # 增加迭代次数


Turn 1
['V8']
0
The max degree is 4 @ V8
[7, 2, 4, 6, 8, 1, 5, 8, 9, 0, 3, 10, 12, 13]
The turn 1 visited node is ['V8', 'V3', 'V5', 'V7', 'V9', 'V2', 'V6', 'V10', 'V1', 'V4', 'V11', 'V13', 'V14']
Still node ['V12', 'V15', 'V16'] not visited
-----------------------------------------------------
Turn 2
['V16']
0
The max degree is 1 @ V16
[15, 11]
The turn 2 visited node is ['V16', 'V12']
Still node ['V15'] not visited
-----------------------------------------------------
Turn 3
-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
Total node ['V8', 'V15', 'V16'] is the origin. 
 3 nodes
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
